🧠 Project Description:
This project is a user-friendly AI-powered tool that automatically generates engaging social media content—captions, hashtags, and emojis—based on keywords or themes provided by users.
It's tailored for platforms like Instagram, LinkedIn, and Twitter, and uses NLP models to help creators, marketers, and individuals craft impactful posts in seconds.
🔧 Built using:
- HuggingFace Transformers
- Gradio (for the interface)
Users can:
- Enter a topic, idea, or keyword
- Choose their target social media platform
- Instantly get optimized captions with emojis and hashtags

🎯 Project Objectives:
- Automate Content Creation
Generate creative, relevant, and high-quality posts with generative AI.
- Platform Optimization
Tailor tone, length, and hashtag style for each platform.
- Boost Engagement
Add expressive emojis and strategic hashtags to boost visibility and sentiment.
- Simple User Interface
Make everything accessible via a Gradio web interface (e.g. on Google Colab).
- Showcase Generative AI in Marketing
Demonstrate how models like GPT-2 can power digital marketing efforts.

Want help writing or testing some sample code for it? I can spin up a prototype interface or help you extend the idea into a full-blown app. Just say the word 🚀



In [8]:
!pip install transformers gradio

In [9]:
import gradio as gr
from transformers import pipeline
import random
import torch

In [10]:
caption_generator = pipeline("text-generation", model="gpt2")

Device set to use cpu


In [11]:
emoji_dict = {
    "positive": ["😊", "😃", "😄", "😁", "😆", "😅", "😂", "🤣", "😍", "😘", "😗", "😙", "😚", "😋", "😜", "😝", "😛", "🤗", "🤩", "🥳", "😇", "🌟"],
    "negative": ["😞", "😟", "😠", "😡", "😔", "😕", "🙁", "☹️", "😣", "😖", "😫", "😩", "😢", "😭", "😤", "😮‍💨", "😱", "😨", "😰", "😥", "😓", "🫌"],
    "neutral":  ["😐", "😑", "😶", "😏", "😒", "🙄", "😬", "🤔", "🤨", "😳", "😞"]
}

sentiment_pipeline = pipeline("sentiment-analysis")

def get_emojis(text):
    label = sentiment_pipeline(text)[0]['label'].lower()
    return ' '.join(random.sample(emoji_dict.get(label, ["😶"]), 3))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [12]:
def get_hashtags(prompt, platform):
    words = prompt.lower().split()
    tags = ["#" + word.replace(" ", "") for word in words if len(word) > 3]

    platform_tags = {
        "Instagram": ["#instadaily", "#igers", "#picoftheday"],
        "LinkedIn": ["#career", "#leadership", "#networking"],
        "Twitter": ["#tweet", "#trending", "#news"]
    }

    return " ".join(tags[:5] + random.sample(platform_tags[platform], 2))

In [13]:
def generate_post(prompt, platform):
    # Generate caption
    caption = caption_generator(prompt, max_length=50, num_return_sequences=1)[0]['generated_text']

    # Generate emojis
    emojis = get_emojis(caption)

    # Generate hashtags
    hashtags = get_hashtags(prompt, platform)

    return caption.strip(), emojis, hashtags

In [14]:
interface = gr.Interface(
    fn=generate_post,
    inputs=[
        gr.Textbox(label="Enter keyword or theme"),
        gr.Radio(["Instagram", "LinkedIn", "Twitter"], label="Choose Platform")
    ],
    outputs=[
        gr.Textbox(label="Generated Caption"),
        gr.Textbox(label="Emojis"),
        gr.Textbox(label="Hashtags")
    ],
    title="Social Media Post & Caption Generator",
    description="Generate catchy captions, relevant hashtags, and emojis based on your theme!"
)

interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2beb43f783580b393c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
